In [97]:
import pandas as pd
import numpy as np
movies=pd.read_excel('movies.xlsx')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8043 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null datetime64[ns]
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj       

In [98]:
movies.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


###### In previous dataset, the below variables include missing data：imbd_id（10），cast（76），homepage（ inv 2936），director（44），tagline（inv 8043），keywords（inv 9373），overview（4），genres（23），production_companies（1030）


###### On the other hand, we find out that almost half of the movie budget and sales number is zero. This is quite abnormal. We are not sure if this is an ordinary phenomenon in movie industry or due to the missing data. It's hard to make decision in the pre-processing period. As our analysis is focused on the factors influencing movie's success, so we will focus on those movies with high profitability, and filter these zero lines in Tableau.

# Question：
1. How does the movie type change across ages?
2. How is the performance of Universal Pictures compared to Paramount Pictures?
3. How is the performance of the movie based on fictions, comparing with those don't?
4. Will the actors and directors influence the audience's comment?

# Variables required
1. Popularity：vote_count,vote_average，popularity
2. Benefit：budget_adj,profit_adj
3. Controlling variable：genres(电影风格),keywords,production_companies,cast,director
4. Other：release_date,release_year

# Data Pre-processing

#### delete the lines with genres missed，one type of genre per line，create a new dataset named "movies_genres"

In [99]:
def add_lines(datalist,col,mark):
    return datalist.drop(col,axis=1).join(datalist[col].str.split(mark,expand=True).stack().reset_index(level=1,drop=True).rename(col))
def delete_null(datalist,col):
    return datalist[datalist[col].notnull()]

movies_genres=add_lines(movies,'genres','|')
movies_genres=delete_null(movies_genres,'genres')

#### delete lines with keywords missed，one keyword per line，create a new dataset named "movies_sellpoint"

In [100]:
movies_sellpoint=add_lines(movies,'keywords','|')
movies_sellpoint=delete_null(movies_sellpoint,'keywords')

#### delete lines with production_companies missed，one company per line，create new dataset named "movies_companies"

In [101]:
movies_companies=add_lines(movies,'production_companies','|')
movies_companies=delete_null(movies_companies,'production_companies')

#### delete lines with cast missed，ont cast per line，create new dataset named "movies_cast"

In [102]:
movies_cast=add_lines(movies,'cast','|')
movies_cast=delete_null(movies_cast,'cast')

#### delete lines with director missed，one director per line，create new dataset named "movies_directors"

In [103]:
movies_directors=add_lines(movies,'director','|')
movies_directors=delete_null(movies_directors,'director')

# Data output

In [105]:
writer=pd.ExcelWriter('movies.xlsx')
movies.to_excel(writer,'raw data')
movies_genres.to_excel(writer,'genres')
movies_sellpoint.to_excel(writer,'keywords')
movies_companies.to_excel(writer,'companies')
movies_cast.to_excel(writer,'cast')
movies_directors.to_excel(writer,'director')
writer.save()